# Scrape big 5
https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats


In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup


In [2]:
# URL = 'https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats'
# We are using last seasons data atm...cuz there arent enough data! season has just started
URL = 'https://fbref.com/en/comps/Big5/2023-2024/stats/players/2023-2024-Big-5-European-Leagues-Stats'

In [3]:
res = Request(URL,headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'http://google.com',  # Optional, mimics the referer if needed
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Connection': 'keep-alive'  # Helps maintain a persistent connection
})
html_page = urlopen(res).read()
soup = BeautifulSoup(html_page, 'html.parser')


In [4]:
table = soup.find('table', {'id': 'stats_standard'})


In [5]:
rows = [tr.get_text() for tr in table.find_all('tr')]


In [6]:
import pandas as pd

In [57]:
df = pd.read_html(html_page)[0]
df.columns = df.columns.droplevel()
# Drop Rk Player Natio etc stuff
df = df[df['Rk'] != 'Rk']
# Drop first RK row cuz we dont really need it
df = df.drop('Rk', axis=1)
# Make index cts
df.reset_index(drop=True, inplace=True)

# Drop Match column cuz they dont do anything atm
df = df.drop('Matches', axis=1)

In [43]:
# So now, last 11 values are per 90 stats...so we have to smh mark that?
df.head()

,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,...,Gls,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG
0,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,23,2000,20,13,1237,...,0.00,0.07,0.07,0.00,0.07,0.00,0.06,0.06,0.00,0.06
1,Brenden Aaronson,us USA,"MF,FW",Union Berlin,de Bundesliga,22,2000,30,14,1267,...,0.14,0.14,0.28,0.14,0.28,0.14,0.13,0.27,0.14,0.27
2,Paxten Aaronson,us USA,MF,Eint Frankfurt,de Bundesliga,19,2003,7,1,101,...,0.00,0.89,0.89,0.00,0.89,0.11,0.07,0.19,0.11,0.19
3,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,17,2006,1,0,4,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,35,1987,31,31,2781,...,0.13,0.00,0.13,0.10,0.10,0.11,0.01,0.12,0.09,0.09


In [62]:
duplicate_columns = df.columns[df.columns.duplicated()]
occured = []
cols = []
count = 1
stat_90s = False
for column in df.columns:
    if not stat_90s:
        if column in duplicate_columns and column in occured:
            # Means we are now in 90s stats
            cols.append(f'{column}-90')
            continue
        else:
            occured.append(column)
            cols.append(column)
    else:
        cols.append(f'{column}-90')

df.columns = cols


In [63]:
df.head()

,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,...,Gls-90,Ast-90,G+A-90,G-PK-90,G+A-PK,xG-90,xAG-90,xG+xAG,npxG-90,npxG+xAG-90
0,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,23,2000,20,13,1237,...,0.00,0.07,0.07,0.00,0.07,0.00,0.06,0.06,0.00,0.06
1,Brenden Aaronson,us USA,"MF,FW",Union Berlin,de Bundesliga,22,2000,30,14,1267,...,0.14,0.14,0.28,0.14,0.28,0.14,0.13,0.27,0.14,0.27
2,Paxten Aaronson,us USA,MF,Eint Frankfurt,de Bundesliga,19,2003,7,1,101,...,0.00,0.89,0.89,0.00,0.89,0.11,0.07,0.19,0.11,0.19
3,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,17,2006,1,0,4,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,35,1987,31,31,2781,...,0.13,0.00,0.13,0.10,0.10,0.11,0.01,0.12,0.09,0.09


In [64]:
df.to_csv('standard_stats_all_5_leagues.csv')